In [1]:
import pandas as pd
import numpy as np
import datetime
import tqdm
import matplotlib.pyplot as plt
import pytictoc
import multiprocessing
from collections import defaultdict, OrderedDict
from tqdm import tqdm_notebook
import itertools
from joblib import Parallel, delayed

In [4]:
prequants = pd.read_csv("../../data/coms_sep/quantiles_data/for_quantiles.csv", index_col=0)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


---

In [5]:
prequants['short_date'] = pd.to_datetime(prequants['event_time']).dt.strftime("%Y%m%d").astype(int)
prequants.drop('event_time', axis=1, inplace=True)

In [ ]:
channels = ['ATMAPI', 'MBK', 'MOBILEAPI', 'UFS', 'WEBAPI', 'amount_original']

In [ ]:
prequants = pd.get_dummies(prequants, columns=['channel_indicator_desc'], prefix="", prefix_sep="")
prequants.rename(columns={'UFS.WEBAPI': 'UFS'}, inplace=True)

for i in channels[:-1]:
    prequants[i] = (prequants["amount_original"] * prequants[i]).astype(int)
prequants["amount_original"] = prequants["amount_original"].astype(int)

# TRAIN

## 30 days, amount + channels

In [ ]:
#говнокод
dates_big_list = \
[int(f'201710%0.2d' % x) for x in list(range(28, 32))] + \
[int(f'201711%0.2d' % x) for x in list(range(1, 31))] + \
[int(f'201712%0.2d' % x) for x in list(range(1, 6))]

In [17]:
def quants_per_channel(channel):
    df = prequants[['user_id', 'short_date', channel]].copy()
    df = df.query(f"{channel} != 0")
    
    quants_range = np.arange(0.1, 1.1, 0.1)
    NDAYS = 30 # за какой временной промежуток берем медиану
    quants_names = ['%s_%s_days_quantile_%0.1f' % (channel, NDAYS, q) for q in quants_range]
    
    
    grouper = itertools.groupby(zip(df['user_id'], df['short_date'], df[channel]), key = lambda x: x[0])
    grouping_result = defaultdict(dict)
    for uid, values in tqdm_notebook(grouper, total = df['user_id'].nunique()):
        (_, dates, amounts) = zip(*values)
        date_amount_dict = defaultdict(list)
        # сначала сделаем dict из даты в качестве ключа и amount каждой транзакции пользователя за этот день
        for date, amount_iter in itertools.groupby(zip(dates, amounts), key = lambda x: x[0]):
            _, day_amount_list = list(zip(*amount_iter))
            date_amount_dict[date] = day_amount_list
        # при правке интервалов даты поправить STARTING_NDAY, NDAYS
        for n in range(len(dates_big_list)):
            calculating_dates = dates_big_list[n - NDAYS: n]
            calculating_list = list(itertools.chain.from_iterable((date_amount_dict[x] for x in calculating_dates)))
            calc_quantile = lambda q: np.percentile(calculating_list, q * 100) if len(calculating_list) else 0
            quantiles = [calc_quantile(q) for q in quants_range]
            grouping_result[uid][dates_big_list[n]] = quantiles
    
    df_grouped_list = [[(uid, date, *quantiles) for date, quantiles in date_dict.items()] \
                       for uid, date_dict in grouping_result.items()]
    df_grouped = pd.DataFrame(list(itertools.chain.from_iterable(df_grouped_list)), 
                              columns = ['user_id', 'short_date'] + quants_names)
    df_grouped.to_csv("../../data/coms_sep/quantiles_data/qnts_train_30_{}.csv".format(channel))
    return 'OK'

In [21]:
%%time
(Parallel(n_jobs=6)(delayed(quants_per_channel)(channel)
                    for channel in channels))







CPU times: user 1.53 s, sys: 540 ms, total: 2.07 s
Wall time: 34min 1s


['OK', 'OK', 'OK', 'OK', 'OK', 'OK']

## 90 days, amount_original
#### не делаем, потому что в тесте нет столько истории

# TEST

## 30 days channels+amount

In [199]:
import os
import re
FIRST_N = 100
N_THREADS = 5
test_folder = '../../data/raw_splits/test/'
test_files = sorted([x for x in os.listdir(test_folder) if not '.pkl' in x], key = lambda x: int(re.sub('[^0-9]', '', x)))
test_files = [os.path.join(test_folder, x) for x in test_files]
print(f'Length of test files is {len(test_files)}')

Length of test files is 26


In [200]:
#говнокод
dates_big_list = \
[int(f'201711%0.2d' % x) for x in list(range(29, 31))] + \
[int(f'201712%0.2d' % x) for x in list(range(1, 6))]

In [201]:
dates_big_list

[20171129, 20171130, 20171201, 20171202, 20171203, 20171204, 20171205]

In [202]:
def get_data(chunk_name):
    chunk_df = pd.read_feather(chunk_name)[['user_id', 'event_time',
                                           'amount_original',
                                           'channel_indicator_desc']]
    return chunk_df

In [203]:
import multiprocessing

In [230]:
%%time
# create features matrix
with multiprocessing.Pool(processes=5) as pool:
    results = pool.map(get_data, test_files)

CPU times: user 3.37 s, sys: 5.26 s, total: 8.64 s
Wall time: 13.3 s


In [231]:
prequants = pd.DataFrame()
for i, df in enumerate(results):
    prequants = pd.concat([prequants, df])

In [232]:
prequants = prequants.loc[np.logical_not(np.isnan(prequants['amount_original']))]

In [169]:
prequants.to_csv("../../data/coms_sep/for_quantiles_test.csv")

In [233]:
prequants['short_date'] = pd.to_datetime(prequants['event_time']).dt.strftime("%Y%m%d").astype(int)
prequants.drop('event_time', axis=1, inplace=True)

In [234]:
prequants.short_date.min(), prequants.short_date.max()

(20171007, 20171205)

train channels = ['ATMAPI', 'MBK', 'MOBILEAPI', 'UFS.WEBAPI', 'WEBAPI']
<br>
unexpected in test = ['CALLCENTER', 'CRM', 'IVR', 'VSP',]

In [235]:
channels = ['ATMAPI', 'MBK', 'MOBILEAPI', 'UFS', 'WEBAPI', 'amount_original']

In [236]:
prequants = pd.get_dummies(prequants, columns=['channel_indicator_desc'], prefix="", prefix_sep="")

In [242]:
prequants.rename(columns={'UFS.WEBAPI': 'UFS'}, inplace=True)

In [243]:
for i in channels[:-1]:
    prequants[i] = (prequants["amount_original"] * prequants[i]).astype(int)
prequants["amount_original"] = prequants["amount_original"].astype(int)

In [305]:
def quants_per_channel(channel):
    df = prequants[['user_id', 'short_date', channel]].copy()
    df = df.query(f"{channel} != 0")
    
    quants_range = np.arange(0.1, 1.1, 0.1)
    NDAYS = 30 # за какой временной промежуток берем квантили
    quants_names = ['%s_%s_days_quantile_%0.1f' % (channel, NDAYS, q) for q in quants_range]
    # assert STARTING_NDAY >= NDAYS
    
    
    
    grouper = itertools.groupby(zip(df['user_id'], df['short_date'], df[channel]), key = lambda x: x[0])
    grouping_result = defaultdict(dict)
    for uid, values in tqdm_notebook(grouper, total = df['user_id'].nunique()):
        (_, dates, amounts) = zip(*values)
        date_amount_dict = defaultdict(list)
        # сначала сделаем dict из даты в качестве ключа и amount каждой транзакции пользователя за этот день
        for date, amount_iter in itertools.groupby(zip(dates, amounts), key = lambda x: x[0]):
            _, day_amount_list = list(zip(*amount_iter))
            date_amount_dict[date] = day_amount_list
        # при правке интервалов даты поправить STARTING_NDAY, NDAYS
        for n in range(len(dates_big_list)):
            calculating_dates = dates_big_list[n - NDAYS: n]
            calculating_list = list(itertools.chain.from_iterable((date_amount_dict[x] for x in calculating_dates)))
            calc_quantile = lambda q: np.percentile(calculating_list, q * 100) if len(calculating_list) else 0
            quantiles = [calc_quantile(q) for q in quants_range]
            grouping_result[uid][dates_big_list[n]] = quantiles
    
    df_grouped_list = [[(uid, date, *quantiles) for date, quantiles in date_dict.items()] \
                       for uid, date_dict in grouping_result.items()]
    df_grouped = pd.DataFrame(list(itertools.chain.from_iterable(df_grouped_list)), 
                              columns = ['user_id', 'short_date'] + quants_names)
    # поменял на test ----
    df_grouped.to_csv("../../data/coms_sep/quantiles_data/qnts_test_30_{}.csv".format(channel))
    return 'OK'

In [306]:
%%time
(Parallel(n_jobs=6)(delayed(quants_per_channel)(channel)
                    for channel in channels))







CPU times: user 1.58 s, sys: 1.56 s, total: 3.14 s
Wall time: 8min 13s


['OK', 'OK', 'OK', 'OK', 'OK', 'OK']